In [1]:
#import dependencies

import numpy as np
#import nltk


In [2]:
from helper import *

# Define tokens' value
GO_ID = 0
EOS_ID = 1
UNK_ID = 2
PAD_ID = 3

In [1]:
english_text_path = "dataset/small_vocab_en.txt"
french_text_path = "dataset/small_vocab_fr.txt"

french_sentences, french_tables = data_pipeline(french_text_path)
english_sentences, english_tables = data_pipeline(english_text_path, padding=True)


NameError: name 'data_pipeline' is not defined

In [4]:
en_max_len, en_min_len = sentence_length(english_sentences)
fr_max_len, fr_min_len = sentence_length(french_sentences)

In [5]:
source_inputs = np.array([tokenize(sentence,en_max_len, english_tables, source=True, reverse=True)\
                          for sentence in english_sentences])
target_outputs = np.array([tokenize(sentence,fr_max_len, french_tables, source=False, reverse=False)\
                          for sentence in french_sentences])

In [6]:
import tensorflow as tf

In [7]:
from seq2seq import seq2seq

In [8]:
# global Hyperparameters
batch_size = 128
time_major = False
num_units = 256
lr = 1e-3
inp_shape = (None, None,)
dtype = tf.int32

In [27]:
hparams = {
    "batch_size":128,
    "time_major":False,
    "num_units" :256,
    "src_embed_dim":300,
    "use_enc_final_state":True,
    "rnn_layers":2,
    "keep_prob":0.8,
    "query_units":250,
    "attn_layer_size":125,
    "lr":1e-3,
    "tgt_embed_dim":300,
          }



In [30]:
import json 
fp = open("hyperparameters.json","w")
json.dump(hparams, fp)
fp.close()

In [10]:

train_graph = tf.Graph()
test_graph = tf.Graph()
infer_graph = tf.Graph()




In [11]:
with train_graph.as_default():
    train_model = seq2seq(mode="train", hparams=hparams)
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [12]:
with test_graph.as_default():
    test_model = seq2seq(mode="test", hparams=hparams)

In [13]:
with infer_graph.as_default():
    infer_model = seq2seq(mode="infer", hparams=hparams)

In [14]:
'''
with train_graph.as_default():
    
    encoder_hparams = dict(batch_size=batch_size, time_major=time_major,
                      inp_shape=inp_shape, dtype=tf.int32,
                      embed_dim=300, vocab_size=english_tables.vocab_size,max_ts=en_max_len,
                      max_time_step=en_max_len, num_units=num_units)
    encoder = Encoder(encoder_hparams)
    encoder.build_model()
    
    
    
    decoder_hparams = dict(batch_size=batch_size, time_major=time_major, num_units=num_units,\
                          learning_rate=1e-3, embed_dim=250, vocab_size=french_tables.vocab_size,
                          memory=encoder.outputs , mem_seq_length=encoder.seq_length,enc_final_state=encoder.final_state,\
                          inp_shape=inp_shape, dtype=tf.int32,attn_units=250, max_ts=fr_max_len)
    decoder = Decoder(decoder_hparams)
    decoder.build_model(mode="train")
    saver = tf.train.Saver()
'''

'\nwith train_graph.as_default():\n    \n    encoder_hparams = dict(batch_size=batch_size, time_major=time_major,\n                      inp_shape=inp_shape, dtype=tf.int32,\n                      embed_dim=300, vocab_size=english_tables.vocab_size,max_ts=en_max_len,\n                      max_time_step=en_max_len, num_units=num_units)\n    encoder = Encoder(encoder_hparams)\n    encoder.build_model()\n    \n    \n    \n    decoder_hparams = dict(batch_size=batch_size, time_major=time_major, num_units=num_units,                          learning_rate=1e-3, embed_dim=250, vocab_size=french_tables.vocab_size,\n                          memory=encoder.outputs , mem_seq_length=encoder.seq_length,enc_final_state=encoder.final_state,                          inp_shape=inp_shape, dtype=tf.int32,attn_units=250, max_ts=fr_max_len)\n    decoder = Decoder(decoder_hparams)\n    decoder.build_model(mode="train")\n    saver = tf.train.Saver()\n'

In [15]:
train_model.dec_out_

<tf.Tensor 'Placeholder_1:0' shape=(128, ?) dtype=int32>

In [16]:
with tf.Session(graph=train_graph) as sess:
    sess.run(init)
   
    for e in range(1, 2):
        for i, (X, y) in enumerate(batch(128, source_inputs[128:], target_outputs[128:])):
            loss = train_model.train_batch(sess, X,y)
            step = i+1
            if step % 50 == 0:
                #cost = sess.run(tf.reduce_mean(cost))
                print("epoch : {} step : {} loss : {}".format(e, i+1, loss))

    saver.save(sess, "model.ckpt")
    
   

epoch : 1 step : 50 loss : 1.8534660339355469
epoch : 1 step : 100 loss : 1.1035282611846924
epoch : 1 step : 150 loss : 0.8263322710990906
epoch : 1 step : 200 loss : 0.6668124794960022
epoch : 1 step : 250 loss : 0.5746607184410095
epoch : 1 step : 300 loss : 0.5122098326683044
epoch : 1 step : 350 loss : 0.4023069143295288
epoch : 1 step : 400 loss : 0.32048729062080383
epoch : 1 step : 450 loss : 0.1924370378255844
epoch : 1 step : 500 loss : 0.10190078616142273
epoch : 1 step : 550 loss : 0.11560405045747757
epoch : 1 step : 600 loss : 0.12727314233779907
epoch : 1 step : 650 loss : 0.07376065850257874
epoch : 1 step : 700 loss : 0.05063146352767944
epoch : 1 step : 750 loss : 0.08060124516487122
epoch : 1 step : 800 loss : 0.0887700617313385
epoch : 1 step : 850 loss : 0.05335487425327301
epoch : 1 step : 900 loss : 0.05560033395886421
epoch : 1 step : 950 loss : 0.045011188834905624
epoch : 1 step : 1000 loss : 0.04666342958807945
epoch : 1 step : 1050 loss : 0.04342617094516754

In [17]:
train_model.train(sess, X,y)

In [18]:
with tf.Session(graph=infer_graph) as sess:
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, "model.ckpt")
    X = sess.run(infer_model.decoder.sample_id, feed_dict={infer_model.enc_inp_:source_inputs[1].reshape(1,-1)})


Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from model.ckpt


In [19]:
tokens2sentence(X[0],french_tables.int2word)

'les etatsunis est generalement froid en juillet et il gele habituellement en novembre PAD_ID PAD_ID PAD_ID PAD_ID PAD_ID PAD_ID PAD_ID PAD_ID EOS_ID'

In [20]:
X[0]

array([245, 178, 311,  86,  77, 175, 192,  10, 285, 240, 273, 175, 138,
         3,   3,   3,   3,   3,   3,   3,   3,   1], dtype=int32)

In [21]:
tokens2sentence([  3.,   3.,  44.,   5., 197.,  19., 157., 135., 172., 167., 150.,
         83.,  19., 125., 181.], english_tables.int2word)

'PAD_ID PAD_ID july horses monkeys apple yellow fall chinese between saw disliked apple have chilly'

In [22]:
tokens2sentence(source_inputs[0], english_tables.int2word)

'PAD_ID PAD_ID april in snowy is it and autumn during quiet sometimes is jersey new'

In [23]:
tokens2sentence(target_outputs[1], french_tables.int2word)

'les etatsunis est generalement froid en juillet et il gele habituellement en novembre PAD_ID PAD_ID PAD_ID PAD_ID PAD_ID PAD_ID PAD_ID PAD_ID EOS_ID'

In [24]:
tokens2sentence(list(reversed(source_inputs[1])), english_tables.int2word)

'the united states is usually chilly during july and it is usually freezing in november'

In [25]:
x = ['le pamplemousse est mon fruit le plus cher mais la banane est la plus aimee' == i for i in french_sentences] 

In [26]:
x

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 